# Artificial Neural Network

## Importing the libraries.

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf

### The following line displays the version of TensorFlow library.

In [5]:
print(tf.__version__)

2.18.0


## Part 1 - Data Preprocessing

### Importing the dataset.

In [8]:
dataset = pd.read_csv("dataset/winequality-white.csv", delimiter = ';')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [9]:
y = y.reshape(-1, 1)
print(y)

[[6]
 [6]
 [6]
 ...
 [6]
 [7]
 [6]]


### Encoding categorical data

In [11]:
# Represents the separation to binary categories ('good' or 'bad' quality) or evaluation by score degree.
# If value is equal to '0', then will be made a score separation above 6.5 or less.
# From the other hand a precision score implementation will be adopt.
implementation = 0

if implementation == 0:
    temp_v = []
    for i in range(y.shape[0]):
        if y[i] > 6.5:
            temp_v.append(1)
        else:
            temp_v.append(0)
    y = np.array(temp_v)
    loss = 'binary_crossentropy'
    output_units = 1
else:
    loss = 'categorical_crossentropy'
    output_units = len(np.unique(y))

    # Change shape of target array, because need to be vertical and not horizontal 1d array for encoding.
    y = y.reshape(-1, 1)
    
    # Encoding categorical data.
    from sklearn.preprocessing import OneHotEncoder
    # Set sparse=False to get a NumPy array.
    encoder = OneHotEncoder(sparse_output = False)
    y = encoder.fit_transform(y)

In [12]:
print(y)

[0 0 0 ... 0 1 0]


### Splitting the dataset into the Training set and Test set.

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [16]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [19]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [21]:
ann.add(tf.keras.layers.Dense(units = 128, activation='relu'))

### Adding the second hidden layer

In [23]:
ann.add(tf.keras.layers.Dense(units = 128, activation='relu'))

### Adding the output layer

In [25]:
ann.add(tf.keras.layers.Dense(units = output_units, activation = 'sigmoid'))
#ann.add(tf.keras.layers.Dense(units = output_units, activation = 'softmax'))

## Part 3 - Training the ANN

### Compiling the ANN

In [28]:
ann.compile(optimizer='adam', loss = loss, metrics=['accuracy'])
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7484 - loss: 0.5103
Epoch 2/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8188 - loss: 0.3923
Epoch 3/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8255 - loss: 0.3709
Epoch 4/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8314 - loss: 0.3573
Epoch 5/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8193 - loss: 0.3666
Epoch 6/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8302 - loss: 0.3444
Epoch 7/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8496 - loss: 0.3238
Epoch 8/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8459 - loss: 0.3348
Epoch 9/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8484 - loss: 0.3249
Epoch 10/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8494 - loss: 0.3178  
Epoch 11/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8610 - loss: 0.3170
Epoch 12/100
123/123 ━━━━━━━━━━━━━━━━━━

### Predicting the Test set results

In [30]:
y_pred = ann.predict(X_test)

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


### Testing Accuracy

In [32]:
from sklearn.metrics import accuracy_score
if implementation == 0:
    y_pred = (y_pred > 0.5)
    test_accuracy = round(accuracy_score(y_test, y_pred), 2)
else:
    y_pred_conv = np.zeros((y_pred.shape[0], y_pred.shape[1]), dtype = float)
    y_pred_conv[np.arange(len(y_pred)), y_pred.argmax(1)] = 1
    test_accuracy = round(accuracy_score(y_test, y_pred_conv), 2)

In [33]:
print(f"Testing Accuracy is equal to {round(test_accuracy * 100, 2)}%")

Testing Accuracy is equal to 86.0%
